In [1]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GITHUB_ACCESS_TOKEN = os.getenv("GITHUB_ACCESS_TOKEN")

In [3]:
def get_github_events(username):
    url = f"https://api.github.com/users/{username}/events"
    headers = {"Authorization": f"token {GITHUB_ACCESS_TOKEN}"}

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors

    events_data = response.json()
    return events_data

# Example usage
events = get_github_events("whanyu1212")
print(events)

[{'id': '39229506704', 'type': 'PushEvent', 'actor': {'id': 107110503, 'login': 'whanyu1212', 'display_login': 'whanyu1212', 'gravatar_id': '', 'url': 'https://api.github.com/users/whanyu1212', 'avatar_url': 'https://avatars.githubusercontent.com/u/107110503?'}, 'repo': {'id': 814128693, 'name': 'whanyu1212/Transcendent', 'url': 'https://api.github.com/repos/whanyu1212/Transcendent'}, 'payload': {'repository_id': 814128693, 'push_id': 18845571905, 'size': 1, 'distinct_size': 1, 'ref': 'refs/heads/hy-dev', 'head': '5aea1d8db120a17dea61fc40879c3622fc8c4381', 'before': '0794b3cfbcbdbe2eb7336f588a4639720d6d9b14', 'commits': [{'sha': '5aea1d8db120a17dea61fc40879c3622fc8c4381', 'author': {'email': '107110503+whanyu1212@users.noreply.github.com', 'name': 'Hy'}, 'message': 'minor font changes', 'distinct': True, 'url': 'https://api.github.com/repos/whanyu1212/Transcendent/commits/5aea1d8db120a17dea61fc40879c3622fc8c4381'}]}, 'public': True, 'created_at': '2024-06-12T12:26:41Z'}, {'id': '392293

In [4]:
for event in events:
        if event['type'] in ['PushEvent', 'WatchEvent']:
            print(event)

{'id': '39229506704', 'type': 'PushEvent', 'actor': {'id': 107110503, 'login': 'whanyu1212', 'display_login': 'whanyu1212', 'gravatar_id': '', 'url': 'https://api.github.com/users/whanyu1212', 'avatar_url': 'https://avatars.githubusercontent.com/u/107110503?'}, 'repo': {'id': 814128693, 'name': 'whanyu1212/Transcendent', 'url': 'https://api.github.com/repos/whanyu1212/Transcendent'}, 'payload': {'repository_id': 814128693, 'push_id': 18845571905, 'size': 1, 'distinct_size': 1, 'ref': 'refs/heads/hy-dev', 'head': '5aea1d8db120a17dea61fc40879c3622fc8c4381', 'before': '0794b3cfbcbdbe2eb7336f588a4639720d6d9b14', 'commits': [{'sha': '5aea1d8db120a17dea61fc40879c3622fc8c4381', 'author': {'email': '107110503+whanyu1212@users.noreply.github.com', 'name': 'Hy'}, 'message': 'minor font changes', 'distinct': True, 'url': 'https://api.github.com/repos/whanyu1212/Transcendent/commits/5aea1d8db120a17dea61fc40879c3622fc8c4381'}]}, 'public': True, 'created_at': '2024-06-12T12:26:41Z'}
{'id': '39229382